## Setup

In [ ]:
import cv2
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
import os.path as osp
import os
import xml.etree.ElementTree as ET
import imutils
import cyvlfeat as vlfeat
from skimage import filters
from skimage.feature import corner_peaks
import pickle
from random import shuffle
from scipy.spatial.distance import cdist

In [ ]:
from voc_eval import *

def evaluate_mAP():
    detpath = '{}.txt'
    annopath = 'datasets/Annotations/{}.xml'
    imagesetfile = 'datasets/ImageSets/val.txt'
    cachedir = 'cache_anno'

    classes = ['waldo']
    meanAP = 0
    for idx, classname in enumerate(classes) :
        rec, prec, ap = voc_eval(detpath, annopath, imagesetfile, classname,
                                                cachedir, ovthresh=0.5, use_07_metric=False)
        meanAP += ap
        print('{}: {}'.format(classname, ap))

    print('meanAP: {}'.format(meanAP/len(classes)))

## Cascade Classifier

In [ ]:
waldo_cascade = cv2.CascadeClassifier('cascade_waldo/classifier/cascade.xml')
wenda_cascade = cv2.CascadeClassifier('cascade_wenda/classifier/cascade.xml')
wizard_cascade = cv2.CascadeClassifier('cascade_wizard/classifier/cascade.xml')

def cascade_detection_waldo(img_number):
    img = imread('datasets/JPEGImages/' + img_number + '.jpg')
    img_grey = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    candidates, reject_levels, level_weights = waldo_cascade.detectMultiScale3(img_grey, 1.03, 3, outputRejectLevels=True)
    for (x,y,w,h) in waldo_candidates:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),10)
        roi_gray = img_grey[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
    
    plt.imshow(img, interpolation='nearest')
    plt.show()
    
    return candidates, level_weights

def cascade_detection_wenda(img_number):
    img = imread('datasets/JPEGImages/' + img_number + '.jpg')
    img_grey = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    candidates, reject_levels, level_weights = wenda_cascade.detectMultiScale3(img_grey, 1.03, 3, outputRejectLevels=True)
    for (x,y,w,h) in candidates:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),10)
        roi_gray = img_grey[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
    
    plt.imshow(img, interpolation='nearest')
    plt.show()
    
    return candidates, level_weights

def cascade_detection_wizard(img_number):
    img = imread('datasets/JPEGImages/' + img_number + '.jpg')
    img_grey = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    candidates, reject_levels, level_weights = wizard_cascade.detectMultiScale3(img_grey, 1.03, 3, outputRejectLevels=True)
    for (x,y,w,h) in candidates:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),10)
        roi_gray = img_grey[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
    
    plt.imshow(img, interpolation='nearest')
    plt.show()
    
    return candidates, level_weights

In [ ]:
image_ids = []

waldo_candidates_all = []
wenda_candidates_all = []
wizard_candidates_all = []

waldo_level_weights = []
wenda_level_weights = []
wizard_level_weights = []

with open('datasets/ImageSets/val.txt') as file:
    for img_id in file.readlines():
        img_id = img_id.rstrip()
        image_ids.append(img_id)
        
        waldo_candidates, level_weights = cascade_detection_waldo(img_id)
        waldo_candidates_all.append(waldo_candidates)
        waldo_level_weights.append(level_weights)
        
        wenda_candidates, level_weights = cascade_detection_wenda(img_id)
        wenda_candidates_all.append(wenda_candidates)
        wenda_level_weights.append(level_weights)
        
        wizard_candidates, level_weights = cascade_detection_wizard(img_id)
        wizard_candidates_all.append(wizard_candidates)
        wizard_level_weights.append(level_weights)
        
print(image_ids)

In [ ]:
with open('my_waldo.txt', 'w+') as output:
    for i in range(len(image_ids)):
        img_id = image_ids[i]
        print(img_id)
        waldo_candidates = waldo_candidates_all[i]
        print(waldo_candidates)
        level_weights = waldo_level_weights[i]
        print(level_weights)
        
        for j in range(len(waldo_candidates)):
            x,y,w,h = waldo_candidates[j]
            conf = level_weights[j][0]
            result = str(img_id) + ' ' + str(conf) + ' ' + str(x) + ' ' + str(y) + ' ' + str(x + w) + ' ' + str(y + h)
            
            if not j == len(waldo_candidates) - 1:
                result += '\n'
            
            print(result)
            output.write(result)
            
with open('my_wenda.txt', 'w+') as output:
    for i in range(len(image_ids)):
        img_id = image_ids[i]
        print(img_id)
        wenda_candidates = wenda_candidates_all[i]
        print(wenda_candidates)
        level_weights = wenda_level_weights[i]
        print(level_weights)
        
        for j in range(len(wenda_candidates)):
            x,y,w,h = wenda_candidates[j]
            conf = level_weights[j][0]
            result = str(img_id) + ' ' + str(conf) + ' ' + str(x) + ' ' + str(y) + ' ' + str(x + w) + ' ' + str(y + h)
            
            if not j == len(wenda_candidates) - 1:
                result += '\n'
            
            print(result)
            output.write(result)
            
with open('my_wizard.txt', 'w+') as output:
    for i in range(len(image_ids)):
        img_id = image_ids[i]
        print(img_id)
        wizard_candidates = wizard_candidates_all[i]
        print(wizard_candidates)
        level_weights = wizard_level_weights[i]
        print(level_weights)
        
        for j in range(len(wizard_candidates)):
            x,y,w,h = wizard_candidates[j]
            conf = level_weights[j][0]
            result = str(img_id) + ' ' + str(conf) + ' ' + str(x) + ' ' + str(y) + ' ' + str(x + w) + ' ' + str(y + h)
            
            if not j == len(wizard_candidates) - 1:
                result += '\n'
            
            print(result)
            output.write(result)

In [ ]:
evaluate_mAP()